In [1]:
import os
download = not os.path.isfile('data_nmnist/nmnist_test.zip')
print(f'{download=}')
#download = True
import tonic
import tonic.transforms as transforms

testset = tonic.datasets.NMNIST(save_to='./data_nmnist', download=download, train=False)
testloader = tonic.datasets.DataLoader(testset,
                                       batch_size=1,
                                       shuffle=True)

events, target = next(iter(testloader))

download=False


In [2]:
%run boilerplate.ipynb

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.18.1

numpy     : 1.19.2
matplotlib: 3.3.2
pyNN      : 0.9.5
neo       : 0.8.0
moviepy   : 1.0.3
imageio   : 2.9.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.0-56-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Hostname: inv-ope-de06

Git hash: f34be83854ac8bf458f81ef778714973cb6e6ef3

Git repo: https://github.com/SpikeAI/2020-12_brainhack_Project7

Git branch: main



In [3]:
testset.sensor_size, testset.ordering

((34, 34), 'xytp')

In [4]:
N_pop = np.prod(testset.sensor_size)
N_pop

1156

In [5]:
events.shape

torch.Size([1, 6044, 4])

Timings are in microseconds

In [6]:
events[0, :, testset.ordering.find("t")]*1e-6

tensor([0.0027, 0.0037, 0.0042,  ..., 0.3069, 0.3072, 0.3076])

In [7]:
index_x = testset.ordering.find("x")
index_y = testset.ordering.find("y")
index_t = testset.ordering.find("t")
index_x, index_y, index_t

(0, 1, 2)

In [8]:
events[0, :, index_x], events[0, :, index_y] 

(tensor([16, 33,  3,  ..., 12, 21, 29], dtype=torch.int32),
 tensor([11, 12, 33,  ..., 20, 10, 21], dtype=torch.int32))

In [9]:
events[0, :, index_x] == events[0, 0, index_x]

tensor([ True, False, False,  ..., False, False, False])

In [10]:
events[0, :, index_y] == events[0, :, index_y]

tensor([True, True, True,  ..., True, True, True])

In [11]:
(events[0, :, index_x] == events[0, :, index_x]) & (events[0, :, index_y] == events[0, :, index_y])

tensor([True, True, True,  ..., True, True, True])

In [12]:
np.where((events[0, :, index_x] == events[0, :, index_x]) & (events[0, :, index_y] == events[0, :, index_y]))[0]

array([   0,    1,    2, ..., 6041, 6042, 6043])

In [13]:
events[0, np.where((events[0, :, index_x] == events[0, :, index_x]) & (events[0, :, index_y] == events[0, :, index_y]))[0], index_t ]

tensor([  2749,   3672,   4153,  ..., 306926, 307186, 307558],
       dtype=torch.int32)

In [14]:
events_np = events.numpy()
cellSourceSpikes = []
for i in range(N_pop):
    row = i//(testset.sensor_size[0])
    col = i%(testset.sensor_size[0])
    spike_idx = np.where((events_np[0, :, index_x] == row) & (events_np[0, :, index_y] == col))[0]
    spike_times = events_np[0, spike_idx, index_t] * 1.e-3 # in milliseconds
    cellSourceSpikes.append(spike_times)

cellSourceSpikes = [list(elem) for elem in cellSourceSpikes]
idx = events[0, 0, index_x]*(testset.sensor_size[0]) + events[0, 0, index_y] # first spike
len(cellSourceSpikes), cellSourceSpikes[idx], np.sum([len(st) for st in cellSourceSpikes])

(1156,
 [2.749, 14.381, 60.264, 65.494, 122.813, 130.022, 136.074, 269.94, 282.056],
 6044)

In [15]:
try:
    import pyNN.spiNNaker as sim
    simulator = 'spinnaker'
except ModuleNotFoundError:
    import pyNN.nest as sim
    simulator = 'nest'

CSAConnector: libneurosim support not available in NEST.
Falling back on PyNN's default CSAConnector.
Please re-compile NEST using --with-libneurosim=PATH


In [16]:
sim.setup(timestep=dt,
          min_delay=1,
          max_delay=100,
          debug=0)

sources = sim.SpikeSourceArray(spike_times=cellSourceSpikes)
spikeSource = sim.Population(N_pop, sources)    
spikeSource.record(['spikes'])

In [17]:
sim.run(simtime=simtime)

350.0

In [18]:
spikeSources  = spikeSource.get_data()#.segments[0].spiketrains
S_spikes = spikeSources.segments[0].spiketrains

In [19]:
sim.end()

In [20]:
st = S_spikes[idx]
st.as_array()

array([  2.749,  14.381,  60.264,  65.494, 122.813, 130.022, 136.074,
       269.94 , 282.056])

In [21]:
def spikelist2numpy(spikelist):
    output_spike = np.zeros((testset.sensor_size[0], testset.sensor_size[1], time_bins))
    for i, spiketrain in enumerate(spikelist):
        row = i//(testset.sensor_size[0])
        col = i%(testset.sensor_size[0])
        for spiketime in spiketrain.as_array():
            # print(int(spiketime))
            output_spike[row, col, int(spiketime)] = 1
    return output_spike

In [22]:
nmnist_spike = spikelist2numpy(S_spikes)
minmax(nmnist_spike)

movie.min()=0.0 - movie.max()=1.0 


In [23]:
show(make_movie(nmnist_spike, label='nmnist_spike'))

In [24]:
nmnist_spike.shape

(34, 34, 350)

In [25]:
nmnist_spike.sum()

6033.0

# NCARS

In [26]:
download = not os.path.isfile('data_ncars/Prophesee_Dataset_n_cars.zip')

testset = tonic.datasets.NCARS(save_to='./data_ncars', download=download, train=False)
testloader = tonic.datasets.DataLoader(testset,
                                       batch_size=1,
                                       shuffle=True)

events, target = next(iter(testloader))

In [27]:
events_np = events.numpy()
cellSourceSpikes = []
simtime = 0
for i in range(N_pop):
    row = i//(testset.sensor_size[0])
    col = i%(testset.sensor_size[0])
    spike_idx = np.where((events_np[0, :, index_x] == row) & (events_np[0, :, index_y] == col))[0]
    spike_times = events_np[0, spike_idx, index_t] * 1.e-3 # in milliseconds
    cellSourceSpikes.append(spike_times)
    simtime = np.max(simtime, np.max(spike_times))

cellSourceSpikes = [list(elem) for elem in cellSourceSpikes]
idx = events[0, 0, index_x]*(testset.sensor_size[0]) + events[0, 0, index_y] # first spike
len(cellSourceSpikes), cellSourceSpikes[idx], np.sum([len(st) for st in cellSourceSpikes])

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
sim.setup(timestep=dt,
          min_delay=1,
          max_delay=100,
          debug=0)

sources = sim.SpikeSourceArray(spike_times=cellSourceSpikes)
spikeSource = sim.Population(N_pop, sources)    
spikeSource.record(['spikes'])

In [ ]:
sim.run(simtime=simtime)

In [ ]:
spikeSources  = spikeSource.get_data()#.segments[0].spiketrains
S_spikes = spikeSources.segments[0].spiketrains

In [ ]:
sim.end()

In [ ]:
st = S_spikes[idx]
st.as_array()

In [ ]:
def spikelist2numpy(spikelist):
    output_spike = np.zeros((testset.sensor_size[0], testset.sensor_size[1], time_bins))
    for i, spiketrain in enumerate(spikelist):
        row = i//(testset.sensor_size[0])
        col = i%(testset.sensor_size[0])
        for spiketime in spiketrain.as_array():
            # print(int(spiketime))
            output_spike[row, col, int(spiketime)] = 1
    return output_spike

In [ ]:
nmnist_spike = spikelist2numpy(S_spikes)
minmax(nmnist_spike)